<a href="https://colab.research.google.com/github/Bharanidharan-M/lora/blob/main/Lora_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install -q transformers datasets peft accelerate bitsandbytes

In [2]:
from transformers import AutoModelForCausalLM,AutoTokenizer,TrainingArguments,Trainer
from peft import get_peft_model,LoraConfig,TaskType
from datasets import Dataset

In [13]:
data={
    "text":[
        "### Question: Which Indian film grossed the highest in Japan and China?\n### Answer: Dangal",
        "### Question:Which Bollywood film crossed ₹1000 crore globally in 2023?\n### Answer:Pathaan",
        "### Question: What is the highest-grossing Indian film of all time?\n### Answer:Baahubali 2: The Conclusion"
    ]
}
dataset=Dataset.from_dict(data)

In [ ]:
model_name="tiiuae/falcon-rw-1b"
tokenizer=AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token=tokenizer.eos_token

In [5]:
def tokenize(batch):
  tokens=tokenizer(batch["text"],truncation=True,padding="max_length",max_length=128)
  tokens["labels"]=tokens["input_ids"].copy()
  return tokens

In [ ]:
tokenized_dataset=dataset.map(tokenize,batched=True)

In [ ]:
model=AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_8bit=True,
    device_map="auto",
)

In [8]:
lora_config=LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["query_key_value"],
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.CAUSAL_LM

)

In [9]:
model=get_peft_model(model,lora_config)
model.print_trainable_parameters()

trainable params: 1,572,864 || all params: 1,313,198,080 || trainable%: 0.1198


In [10]:
training_args=TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=1,
    learning_rate=2e-4,
    logging_steps=1,
    save_strategy="no",
    report_to="none"
)

In [11]:
trainer=Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer
)
trainer.train()

/tmp/ipython-input-11-1041140387.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer=Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
1,5.333900
2,4.631100
3,4.606200
4,4.262800
5,3.764000
6,2.717000
7,2.985300
8,2.049900
9,2.439600


TrainOutput(global_step=9, training_loss=3.6433224148220487, metrics={'train_runtime': 6.7423, 'train_samples_per_second': 1.335, 'train_steps_per_second': 1.335, 'total_flos': 8364732973056.0, 'train_loss': 3.6433224148220487, 'epoch': 3.0})

In [12]:
model.save_pretrained("lora-finetuned")
tokenizer.save_pretrained("lora-fintuned")

('lora-fintuned/tokenizer_config.json',
 'lora-fintuned/special_tokens_map.json',
 'lora-fintuned/vocab.json',
 'lora-fintuned/merges.txt',
 'lora-fintuned/added_tokens.json',
 'lora-fintuned/tokenizer.json')